In [1]:
import requests
import json
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



In [2]:
#load in shot df and player df
df_shot = pd.read_excel("/Users/lb/Desktop/analytics/w_analytics/new_dtwom_22/shot_final.xlsx")
df_player = pd.read_excel("/Users/lb/Desktop/analytics/w_analytics/new_dtwom_22/2022_players.xlsx")

#merge dataframes only keeping df_shot rows if the player is in df_player
df_total = pd.merge(df_shot, df_player, how="right")
df_total.to_excel("/Users/lb/Desktop/analytics/w_analytics/new_dtwom_22/shot_total.xlsx")

#print(df_shot)

In [27]:
#############UPDATE############
url = "https://data.wnba.com/data/10s/v2015/json/mobile_teams/wnba/2022/scores/pbp/1022200217_full_pbp.json"
r = requests.get(url)
game_data = r.json()

#this function will find the team pids
def team_pids(game_dict):
    away_team_id = 0
    home_team_id = 0
    
    code = game_dict["g"]["gcode"]
    if code[-6:-3] == "ATL":
        away_team_id = 1611661330
    elif code[-6:-3] == "CHI":
        away_team_id = 1611661329
    elif code[-6:-3] == "CON":
        away_team_id = 1611661323        
    elif code[-6:-3] == "DAL":
        away_team_id = 1611661321        
    elif code[-6:-3] == "IND":
        away_team_id = 1611661325
    elif code[-6:-3] == "MIN":
        away_team_id = 1611661324        
    elif code[-6:-3] == "LAS":
        away_team_id = 1611661320        
    elif code[-6:-3] == "LVA":
        away_team_id = 1611661319
    elif code[-6:-3] == "NYL":
        away_team_id = 1611661313        
    elif code[-6:-3] == "PHO":
        away_team_id = 1611661317
    elif code[-6:-3] == "SEA":
        away_team_id = 1611661328        
    elif code[-6:-3] == "WAS":
        away_team_id = 1611661322 
        
    if code[-3:] == "ATL":
        home_team_id = 1611661330
    elif code[-3:] == "CHI":
        home_team_id = 1611661329
    elif code[-3:] == "CON":
        home_team_id = 1611661323        
    elif code[-3:] == "DAL":
        home_team_id = 1611661321        
    elif code[-3:] == "IND":
        home_team_id = 1611661325
    elif code[-3:] == "MIN":
        home_team_id = 1611661324        
    elif code[-3:] == "LAS":
        home_team_id = 1611661320        
    elif code[-3:] == "LVA":
        home_team_id = 1611661319
    elif code[-3:] == "NYL":
        home_team_id = 1611661313        
    elif code[-3:] == "PHO":
        home_team_id = 1611661317
    elif code[-3:] == "SEA":
        home_team_id = 1611661328        
    elif code[-3:] == "WAS":
        home_team_id = 1611661322
    
    return away_team_id, home_team_id
        
away_team_id, home_team_id = team_pids(game_data)


KeyError: 'g'

In [6]:
#print(game_data)

In [24]:
#######for away team########
def starterfnt(game_dict, team_id):
    
    master_list = []
    quarter_list = []
    #find if game went to overtime to modify quarter list
    if len(game_dict["g"]["pd"]) == 7:
        quarter_list = [1,2,3,4,5,6,7]
    elif len(game_dict["g"]["pd"]) == 6:
        quarter_list = [1,2,3,4,5,6]
    elif len(game_dict["g"]["pd"]) == 5:
        quarter_list = [1,2,3,4,5]
    else:
        quarter_list = [1,2,3,4]

    for quarter in quarter_list:
        floor_pids = []
        sub_out = []
        sub_in = []
        for event_dict in game_dict['g']["pd"]:
            if event_dict["p"] == quarter: #only include a specific quarter here
                for event in event_dict["pla"]: #iterates through all events in the quarter
                    if event["tid"] == team_id: #only include away_team events, then home_team events
                        if "Substitution" in event["de"]:
                            sub_out.append(event["pid"]) #will add all pids from away/home team who subbed out
                            sub_in.append(int(event["epid"])) #will add all pids from away/home team who subbed in
        #this for loop will add in players who didn't get subbed out or in but were on the floor the whole time
                        #this will find players who didnt sub in or out but were playing as we can see from their events
                for event in event_dict["pla"]:
                    if event["tid"] == team_id: #only include away_team events, then home_team events            
                        if "Shot" in event["de"] or "shot" in event["de"] or "Free Throw" in event["de"] or "Rebound" in event["de"] or "Foul" in event["de"] or "Assist" in event["de"] or "Turnover" in event["de"]:
                            if event["pid"] not in sub_out and event["pid"] not in sub_in and event["pid"]!= 0 and event["pid"] not in floor_pids:
                                floor_pids.append(event["pid"]) #will add all pids from away/home team who subbed out
                            #this block of code will catch ppl who only assist when not subbing the whole quarter
                            elif event["epid"] != "" and int(event["epid"]) != 0 and int(event["epid"]) not in sub_out and int(event["epid"]) not in sub_in and int(event["epid"]) not in floor_pids:
                                floor_pids.append(int(event["epid"]))

        both = set(sub_out).intersection(sub_in) #makes a set of pids which both subbed out and subbed in
        #print(sub_out, sub_in)
        both_list = list(both) #converts this set to a list
        index_out = [sub_out.index(x) for x in both] #gives index of pid in sub_out if in "both"
        index_in = [sub_in.index(x) for x in both]  #gives index of pid in sub_in if in "both"
        #print(both_list, index_out, index_in)
        
        #append player to floor_pids if player subbed_out before subbing in
        for i in range(len(both_list)):
            if index_out[i] < index_in[i]:
                floor_pids.append(both_list[i])

        #append player to floor_pids if player subbed out but didnt sub in
        for player_id in sub_out: 
            if player_id not in sub_in:
                floor_pids.append(player_id)
                
        master_list.append(floor_pids)
                            
        
    return master_list

away_starters = starterfnt(game_data, away_team_id)
home_starters = starterfnt(game_data, home_team_id)





def total_starters(away_starters, home_starters):
    starter_list = []
    for i in range(len(away_starters)):
        starter_list.append(away_starters[i] + home_starters[i])
    return starter_list
game_starters = total_starters(away_starters, home_starters)






#get all events with teams
def event_list(game_dict, team_id, starter_list):

    total_list = []
    quarter_list = []
    
    if len(game_dict["g"]["pd"]) == 7:
        quarter_list = [1,2,3,4,5,6,7]
    elif len(game_dict["g"]["pd"]) == 6:
        quarter_list = [1,2,3,4,5,6]
    elif len(game_dict["g"]["pd"]) == 5:
        quarter_list = [1,2,3,4,5]
    else:
        quarter_list = [1,2,3,4]
    
    for quarter in quarter_list:
        event_list = []
        floor_pids = []
        for event_dict in game_dict['g']["pd"]:
            if event_dict["p"] == quarter:
                floor_pids = starter_list[quarter-1]
                #print(floor_pids)
                for index, event in enumerate(event_dict["pla"]):
                    
                    if "Substitution" in event["de"]:
                        new_pids = floor_pids.copy()
                    #print(floor_pids)
                    #find the index of player coming out
                        sub_index = new_pids.index(int(event["pid"]))
                    #remove player from list
                        new_pids.remove(int(event["pid"]))
                    #insert new player in their spot
                        new_pids.insert(sub_index, int(event["epid"]))
                        floor_pids = new_pids
                    
                    #print(floor_pids)
                    if event["tid"] == team_id:

                        #making sure the shot was not blocked with this line of code
                        #a blocked shot will have missed in event["de"] and event["opid"] == team id number
                        if "Missed" not in event["de"] or event["opid"] == '':

                            ##############################################
                            if "Shot" in event["de"] or "shot" in event["de"]:

                                if "3pt" in event["de"]:

                                    #3 rebs, 1 block, (3-pointer)
                                    #offensive rebound signified by ["opt1"] ==1 and ["etype"] == 4
                                    if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!='' or event_dict["pla"][index-6]["opid"]!=''):
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 2, floor_pids])

                                    #3 rebs, no blocks (3-pointer)
                                    elif event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 3, floor_pids])

                                    #2 rebs 1 block (3-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!=''):                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])


                                    #2 rebs no blocks (3-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 2, floor_pids])

                                    #1 reb but it went out of bounds or team called timeout or foul
                                    elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])


                                    #1 rebs no blocks (3-pointer)
                                    elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-2]["opid"]=='':                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])

                                #no rebs no blocks or 1 reb 1 block (3-pointer)
                                    else:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 0, floor_pids])                                  

                            ############################# 2-pointers#########
                                elif "Clock" not in event["de"]:
                                    #offensive rebound signified by ["opt1"] ==1 and ["etype"] == 4
                                    #3 rebs, 1 block, (2-pointer)
                                    if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!='' or event_dict["pla"][index-6]["opid"]!=''):
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 2, floor_pids])

                                    #3 rebs, no blocks (2-pointer)
                                    elif event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 3, floor_pids])

                                    #2 rebs 1 block (2-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!=''):                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])


                                    #2 rebs no blocks (2-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 2, floor_pids])

                                    #1 reb but it went out of bounds or team called timeout
                                    elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])


                                    #1 rebs no blocks (2-pointer)
                                    elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-2]["opid"]=='':                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])

                                #no rebs no blocks or 1 reb 1 block (3-pointer)
                                    else:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 0, floor_pids])    




                            ####################################
                            elif "Free Throw" in event["de"]:

                                #free throw after 3 rebounds
                                if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 3, floor_pids])

                                #free throw after 2 rebs               
                                elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 2, floor_pids])

                                #1 reb but it went out of bounds or team called timeout
                                elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"],1, 1, floor_pids])                            

                                #free throw after 1 reb           
                                elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-1]["pid"]!= 0:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 1, floor_pids])                                

                                else:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 0, floor_pids])
                            

        total_list.append(event_list)
    return total_list


away_team_event_list = event_list(game_data, away_team_id, game_starters)




In [64]:
#print(away_team_event_list)

In [25]:
####for home team#####
def starterfnt(game_dict, team_id):
    
    master_list = []
    quarter_list = []
    if len(game_dict["g"]["pd"]) == 7:
        quarter_list = [1,2,3,4,5,6,7]
    elif len(game_dict["g"]["pd"]) == 6:
        quarter_list = [1,2,3,4,5,6]
    elif len(game_dict["g"]["pd"]) == 5:
        quarter_list = [1,2,3,4,5]
    else:
        quarter_list = [1,2,3,4]

    for quarter in quarter_list:
        floor_pids = []
        sub_out = []
        sub_in = []
        for event_dict in game_dict['g']["pd"]:
            if event_dict["p"] == quarter: #only include a specific quarter here
                for event in event_dict["pla"]: #iterates through all events in the quarter
                    if event["tid"] == team_id: #only include away_team events, then home_team events
                        if "Substitution" in event["de"]:
                            sub_out.append(event["pid"]) #will add all pids from away/home team who subbed out
                            sub_in.append(int(event["epid"])) #will add all pids from away/home team who subbed in
        #this for loop will add in players who didn't get subbed out or in but were on the floor the whole time
                        #this will find players who didnt sub in or out but were playing as we can see from their events
                for event in event_dict["pla"]:
                    if event["tid"] == team_id: #only include away_team events, then home_team events            
                        if "Shot" in event["de"] or "shot" in event["de"] or "Free Throw" in event["de"] or "Rebound" in event["de"] or "Foul" in event["de"] or "Assist" in event["de"]:
                            if event["pid"] not in sub_out and event["pid"] not in sub_in and event["pid"]!= 0 and event["pid"] not in floor_pids:
                                floor_pids.append(event["pid"]) #will add all pids from away/home team who subbed out
                            #this block of code will catch ppl who only assist when not subbing the whole quarter
                            elif event["epid"] != "" and int(event["epid"]) != 0 and int(event["epid"]) not in sub_out and int(event["epid"]) not in sub_in and int(event["epid"]) not in floor_pids:
                                floor_pids.append(int(event["epid"]))

        both = set(sub_out).intersection(sub_in) #makes a set of pids which both subbed out and subbed in
        #print(sub_out, sub_in)
        both_list = list(both) #converts this set to a list
        index_out = [sub_out.index(x) for x in both] #gives index of pid in sub_out if in "both"
        index_in = [sub_in.index(x) for x in both]  #gives index of pid in sub_in if in "both"
        #print(both_list, index_out, index_in)
        
        #append player to floor_pids if player subbed_out before subbing in
        for i in range(len(both_list)):
            if index_out[i] < index_in[i]:
                floor_pids.append(both_list[i])

        #append player to floor_pids if player subbed out but didnt sub in
        for player_id in sub_out: 
            if player_id not in sub_in:
                floor_pids.append(player_id)
                
        master_list.append(floor_pids)
                            
        
    return master_list

away_starters = starterfnt(game_data, away_team_id)
home_starters = starterfnt(game_data, home_team_id)





def total_starters(away_starters, home_starters):
    starter_list = []
    for i in range(len(away_starters)):
        starter_list.append(away_starters[i] + home_starters[i])
    return starter_list
game_starters = total_starters(away_starters, home_starters)






#get all events with teams
def event_list(game_dict, team_id, starter_list):

    total_list = []
    quarter_list = []
    
    if len(game_dict["g"]["pd"]) == 7:
        quarter_list = [1,2,3,4,5,6,7]
    elif len(game_dict["g"]["pd"]) == 6:
        quarter_list = [1,2,3,4,5,6]
    elif len(game_dict["g"]["pd"]) == 5:
        quarter_list = [1,2,3,4,5]
    else:
        quarter_list = [1,2,3,4]
    
    for quarter in quarter_list:
        event_list = []
        floor_pids = []
        for event_dict in game_dict['g']["pd"]:
            if event_dict["p"] == quarter:
                #will first populate floor pids with the starters found in the starter function
                floor_pids = starter_list[quarter-1]
                for index, event in enumerate(event_dict["pla"]):
                    
                    if "Substitution" in event["de"]:
                        new_pids = floor_pids.copy()
                    #print(floor_pids)
                    #find the index of player coming out
                        sub_index = new_pids.index(int(event["pid"]))
                    #remove player from list
                        new_pids.remove(int(event["pid"]))
                    #insert new player in their spot
                        new_pids.insert(sub_index, int(event["epid"]))
                        floor_pids = new_pids
                    
                    if event["tid"] == team_id:

                        #making sure the shot was not blocked with this line of code
                        #a blocked shot will have missed in event["de"] and event["opid"] == team id number
                        if "Missed" not in event["de"] or event["opid"] == '':

                            ##############################################
                            if "Shot" in event["de"] or "shot" in event["de"]:

                                if "3pt" in event["de"]:

                                    #3 rebs, 1 block, (3-pointer)
                                    #offensive rebound signified by ["opt1"] ==1 and ["etype"] == 4
                                    if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!='' or event_dict["pla"][index-6]["opid"]!=''):
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 2, floor_pids])

                                    #3 rebs, no blocks (3-pointer)
                                    elif event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 3, floor_pids])

                                    #2 rebs 1 block (3-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!=''):                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])


                                    #2 rebs no blocks (3-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 2, floor_pids])

                                    #1 reb but it went out of bounds or team called timeout or foul
                                    elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])


                                    #1 rebs no blocks (3-pointer)
                                    elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-2]["opid"]=='':                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 1, floor_pids])

                                #no rebs no blocks or 1 reb 1 block (3-pointer)
                                    else:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 3, 0, floor_pids])                                  

                            ############################# 2-pointers#########
                                elif "Clock" not in event["de"]:
                                    #offensive rebound signified by ["opt1"] ==1 and ["etype"] == 4
                                    #3 rebs, 1 block, (2-pointer)
                                    if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!='' or event_dict["pla"][index-6]["opid"]!=''):
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 2, floor_pids])

                                    #3 rebs, no blocks (2-pointer)
                                    elif event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 3, floor_pids])

                                    #2 rebs 1 block (2-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and (event_dict["pla"][index-2]["opid"]!='' or event_dict["pla"][index-4]["opid"]!=''):                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])


                                    #2 rebs no blocks (2-pointer)
                                    elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:                                   
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 2, floor_pids])

                                    #1 reb but it went out of bounds or team called timeout
                                    elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])


                                    #1 rebs no blocks (2-pointer)
                                    elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-2]["opid"]=='':                                 
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 1, floor_pids])

                                #no rebs no blocks or 1 reb 1 block (3-pointer)
                                    else:
                                        event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 2, 0, floor_pids])    




                            ####################################
                            elif "Free Throw" in event["de"]:

                                #free throw after 3 rebounds
                                if event_dict["pla"][index-5]["opt1"] == 1 and event_dict["pla"][index-5]["etype"] == 4 and event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 3, floor_pids])

                                #free throw after 2 rebs               
                                elif event_dict["pla"][index-3]["opt1"] == 1 and event_dict["pla"][index-3]["etype"] == 4 and event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 2, floor_pids])

                                #1 reb but it went out of bounds or team called timeout
                                elif event_dict["pla"][index-2]["opt1"] == 1 and event_dict["pla"][index-2]["etype"] == 4 and ("Stoppage: Out-of-Bounds" in event_dict["pla"][index-1]["de"] or "Team Timeout" in event_dict["pla"][index-1]["de"] or "Foul" in event_dict["pla"][index-1]["de"]):                                 
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"],1, 1, floor_pids])                            

                                #free throw after 1 reb           
                                elif event_dict["pla"][index-1]["opt1"] == 1 and event_dict["pla"][index-1]["etype"] == 4 and event_dict["pla"][index-1]["pid"]!= 0:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 1, floor_pids])                                

                                else:
                                    event_list.append([(((event["locX"]**2 + event["locY"]**2)**.5 )/10.05), event["pid"], 1, 0, floor_pids]) 
                                    

        total_list.append(event_list)
    return total_list


########UPDATE############
home_team_event_list = event_list(game_data, home_team_id, game_starters)



In [66]:
#print (home_team_event_list)

In [26]:
#this fnt will record each scoring event's prob of success, point value, and orebs preceding it

def event_probabilities(team_event_list, df_league):
    #list with all event probs, point values, and orebs preceding it
    team_event_prob_list = [] 
    for quarter_list in team_event_list:
        for event in quarter_list:
                #print (event[1])
                if event[0] == 7.960199004975124:
                    team_event_prob_list.append([float(df_league.loc[df_league["pid"]==event[1], "ft%"].iloc[0]), event[2] , event[3], event[4] ])
                elif event[0] < 5:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "under_5_%"].iloc[0]), event[2] , event[3], event[4]])  
                elif event[0] < 9.5 and event[0] > 5:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "5_9_%"].iloc[0]), event[2] , event[3], event[4]])                   
                elif event[0] < 14.5 and event[0] > 9.5:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "10_14_%"].iloc[0]), event[2] , event[3], event[4]])     
                elif event[0] < 19.5 and event[0] > 14.5:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "15_19_%"].iloc[0]), event[2] , event[3], event[4]])  
                elif event[0] < 24 and event[0] > 19.5:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "20_24_%"].iloc[0]), event[2] , event[3], event[4]])
                elif event[0] > 24:
                    team_event_prob_list.append([(df_league.loc[df_league["pid"]==event[1], "over_24_%"].iloc[0]), event[2] , event[3], event[4]])
        
    return team_event_prob_list


#this fnt will simulate each team's scoring based on events
def expected_points(team_event_prob_list):
    xp_list = []
    
    for index, event in enumerate(team_event_prob_list):
        ####tally up points for no previous rebounds#####
        if event[2] == 0:
            xp_list.append([event[0]*event[1], event[3]])

#tally up points for one prior oreb...will discount it
        if event[2] == 1:
            xp_list.append([event[0] * event[1] * (1-(team_event_prob_list[index-1][0])), event[3]])


#tally up points for two prior orebs...will discount it
        if event[2] == 2:
            xp_list.append( [event[0] * event[1] * ((1-(team_event_prob_list[index-2][0]))  * (1-(team_event_prob_list[index-1][0]))), event[3]])


#tally up points for three prior orebs...will discount it
        if event[2] == 3:
            xp_list.append([event[0] * event[1] * ((1-(team_event_prob_list[index-3][0]))  * (1-(team_event_prob_list[index-2][0]))  * (1-(team_event_prob_list[index-1][0]))), event[3]])

    return xp_list
#this will return a list of [xP, [list of players to be credited], list of players to be debited]"""


#team a event probs
away_team_event_prob_list = event_probabilities(away_team_event_list, df_total)
#print(away_team_event_prob_list)

#team b event probs
home_team_event_prob_list = event_probabilities(home_team_event_list, df_total)
#print(home_team_event_prob_list)

away_team_xp_list = expected_points(away_team_event_prob_list)
#print (away_team_xp_list)

home_team_xp_list = expected_points(home_team_event_prob_list)
#print (away_team_xp_list)

def xp_player_for(away_xp, home_xp):
    
    #first column is xpfor, second is xpagainst, third is xp percentage
    xp_dict = {}
    
    #fill dictionary with all players
    for event in away_xp:
        for player in event[1]:
            #print(player)
            if player not in xp_dict:
                xp_dict[player] = [0,0,0]
    #this line of code is just to find home players who were only on the floor for shots for...unlikely, I know, but technically possible
    for event in home_xp:
        for player in event[1]:
            #print(player)
            if player not in xp_dict:
                xp_dict[player] = [0,0,0]
     
    #credit away and then home team players with xp
    for event in away_xp:
        for player in event[1][0:5]:
            xp_dict[player][0] += event[0]
    for event in home_xp:
        for player in event[1][5:10]:
            xp_dict[player][0] += event[0]
            
    #debit away and then home team players with xp
    for event in home_xp:
        for player in event[1][0:5]:
            xp_dict[player][1] -= event[0]
    for event in away_xp:
        for player in event[1][5:10]:
            xp_dict[player][1] -= event[0]
            
    for key in xp_dict:
        xp_dict[key][2] = round((xp_dict[key][0]/(xp_dict[key][0]- (xp_dict[key][1]))*100),2)
    
    df = pd.DataFrame(xp_dict.values(), columns = ["xPF", "xPA", "xP%"], index=xp_dict.keys())
    
    
    return df

df = xp_player_for(away_team_xp_list, home_team_xp_list) 

df

,xPF,xPA,xP%
1631007,47.253134,-54.485001,46.45
203866,57.084275,-52.731615,51.98
1629497,70.323501,-78.551399,47.24
1629479,45.156557,-38.721310,53.84
1628277,64.418723,-56.708315,53.18
1631135,54.798804,-57.611338,48.75
1629478,50.584907,-45.388704,52.71
1628279,75.671325,-68.183496,52.60
1628886,37.879621,-33.196492,53.29
1628882,28.775189,-38.251177,42.93


ValueError: 1629546 is not in list

In [10]:
total = 0
for event in home_team_xp_list:
    total += event[0]
print(total)

71.71746709252406


In [11]:
print(df["xPF"].sum() + df["xPA"].sum())

-2.2737367544323206e-13
